In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data and display first 5 rows
z_data = pd.read_csv('esem_act_deact_zscore_groups.csv',sep=';',decimal='.')
z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)
data = z_data

#we need the list with the labels to test the predictions; everytime the label or proband changes, we add it to the list
# grouped = z_data.groupby(['proband', 'trial','task'])
# groupedAgg = grouped.aggregate(np.mean)
labelsComputingAccuracy = z_data.groupby(['proband', 'trial','task']).aggregate(np.mean).index.get_level_values(level='task')
labels = z_data.task

#=== remove everything that is not a feature ===================================================
z_data = z_data.drop('proband', axis = 1)
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('task', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = z_data.columns

# Convert to numpy array
features = np.array(z_data)
labels = np.array(labels)

numLabels = 3 # num of different labels (comprehension and rest)
sizeTrainSet = 13 # num of participants used for training
sizeTestSet = 4 # num participants used for testing
 
# get the number of trials per participant
numScans = len(data.scan.unique())
numParticipants = sizeTrainSet + sizeTestSet


#get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
training_features = features[0:numScans*sizeTrainSet]
training_features = np.array(training_features)
testing_features = features[numScans*sizeTrainSet:numScans*numParticipants]
testing_features = np.array(testing_features)

training_target = labels[0:numScans*sizeTrainSet]
training_target = np.array(training_target)
testing_target = labels[numScans*sizeTrainSet:numScans*numParticipants]
testing_target = np.array(testing_target)


In [4]:
print(training_features.shape)
print(testing_features.shape)
print(training_target.shape)
print(testing_target.shape)
print(len(labelsComputingAccuracy))
print(len(z_data))
print(numScans)
z_data.head(5)

(8723, 1139)
(2668, 1139)
(8723,)
(2668,)
1020
11391
671


,140-99-73_act,140-102-73_act,140-105-73_act,140-108-73_act,140-99-76_act,140-102-76_act,140-105-76_act,140-108-76_act,140-99-79_act,140-102-79_act,...,137-186-103_deact,137-189-103_deact,137-180-106_deact,137-183-106_deact,137-186-106_deact,137-189-106_deact,137-186-109_deact,140-180-103_deact,140-183-103_deact,140-183-106_deact
0,-1.161421,-0.923320,-0.320038,0.629510,-0.136895,-0.536765,-0.514446,-0.463186,-0.502790,-0.892422,...,1.535653,1.118526,0.601662,1.391459,2.346925,1.726898,1.795029,0.146092,0.628055,0.868627
1,-1.257476,-0.909639,-0.472218,0.218840,-0.140112,-0.213969,-0.352091,-0.384465,-0.116147,-0.223085,...,0.624662,0.490574,0.227359,0.635839,1.355147,0.749432,1.385217,-0.416112,0.042862,0.044537
2,-0.537064,-0.300862,-0.773474,-1.240208,0.297384,0.712767,0.028945,-0.170796,0.775871,1.227146,...,0.141350,0.098104,-0.154678,0.083427,0.491239,-0.052157,0.857380,-0.530432,-0.058034,-0.589379
3,-0.387332,-0.971201,-1.658605,-1.580764,-0.272004,-1.237891,-1.372606,-0.676855,0.431850,0.328499,...,0.516873,0.299945,-0.300068,0.189964,0.775050,0.526205,0.916393,-0.228189,-0.002541,-0.394133
4,-0.000287,-0.454766,-1.282812,-1.347049,0.593337,-0.040423,-0.517759,-0.455688,1.549156,1.456456,...,-0.940016,-1.127898,-1.047128,-0.784648,-0.678311,-0.309207,-0.729411,-0.881222,-1.041764,-1.274007


In [3]:
from tpot import TPOTClassifier
fileNameTPot = 'AllROIs/tpot_mnist_pipeline_NotAggregatedParticipantSplit_AllROIs'

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs = 20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

Optimization Progress:  55%|█████▌    | 66/120 [20:18<7:20:15, 489.17s/pipeline] 

Generation 1 - Current best internal CV score: 0.5680377750322021


Optimization Progress:  78%|███████▊  | 93/120 [28:26<2:53:45, 386.15s/pipeline]

Generation 2 - Current best internal CV score: 0.5680377750322021


Optimization Progress:  96%|█████████▌| 115/120 [2:02:09<2:36:19, 1875.96s/pipeline]

Generation 3 - Current best internal CV score: 0.5782403196551089


Optimization Progress: 138pipeline [2:07:26, 1014.07s/pipeline]                     

Generation 4 - Current best internal CV score: 0.5796198838095739


Generation 5 - Current best internal CV score: 0.5796198838095739

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.35000000000000003, min_samples_leaf=17, min_samples_split=3, n_estimators=100)
Imputing missing values in feature set


ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [ ]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [ ]:
%run -i $fileForLearning

In [ ]:
z_data = data
condition = z_data.at[sizeTrainSet*numScans, 'task']
participant = z_data.at[sizeTrainSet*numScans, 'proband']
majorityVote = []
countComprehension = 0
countRest = 0
countD2 = 0
    
for i,j in zip(range(sizeTrainSet*numScans,len(z_data)),range(len(results))):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
        if (countComprehension > countRest) and (countComprehension > countD2):
            majorityVote.append('C')
        elif countD2 > countRest and countD2 > countComprehension:
                majorityVote.append('D2')
        elif countRest > countComprehension and countRest > countD2:
            majorityVote.append('R')
        elif countComprehension == countRest:
            majorityVote.append('CR')
        elif countComprehension == countD2:
            majorityVote.append('CD2')
        else:
            majorityVote.append('D2R')
        countComprehension = 0
        countRest = 0
        countD2 = 0

    if (results[j] == 0):
        countComprehension = countComprehension + 1
    if (results[j] == 1):
        countRest = countRest + 1
    if (results[j] == 2):
        countD2 = countD2 + 1

if (results[j] == 0):
    countComprehension = countComprehension + 1
if (results[j] == 1):
    countRest = countRest + 1
if (results[j] == 2):
    countD2 = countD2 + 1

if (countComprehension > countRest) and (countComprehension > countD2):
    majorityVote.append('C')
elif countD2 > countRest and countD2 > countComprehension:
        majorityVote.append('D2')
elif countRest > countComprehension and countRest > countD2:
    majorityVote.append('R')
elif countComprehension == countRest:
    majorityVote.append('CR')
elif countComprehension == countD2:
    majorityVote.append('CD2')
else:
    majorityVote.append('D2R')


In [ ]:
labelsComputingAccuracy = pd.Series(labelsComputingAccuracy)
labelsComputingAccuracy = labelsComputingAccuracy.replace({0: 'C', 1: 'R', 2: 'D2'})

num_matches = 0;
for a, b in zip(majorityVote, labelsComputingAccuracy):
    if b in a:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',len(labelsComputingAccuracy),')')

accuracy = num_matches/len(labelsComputingAccuracy)*100
print('Accuary: ',accuracy)


In [ ]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'NoAgg_AllROIs' + str(accuracy) + '\n')
